In [13]:
from datasets import load_dataset
import pandas as pd
import logging
import time
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
import os
from langchain_cohere import ChatCohere

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from IPython.display import display, HTML
import json

In [14]:
file_path = 'pes_arab_dataset.csv'  # Path for the CSV file
data = pd.read_csv(file_path)

# Sample 50 entries from the dataset
sampled_data = data.sample(n=5, random_state=42)

sampled_data

,link,question_number,flores_passage,question,mc_answer1,mc_answer2,mc_answer3,mc_answer4,correct_answer_num,dialect,ds
70,https://en.wikibooks.org/wiki/High_School_Eart...,1,بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی...,کدام یک از موارد زیر را انسان‌ها در قرن شانزده...,استفاده از عدسی‌ها,بزرگنمایی اشیاء,ساخت عدسی‌ها,ساخت تلسکوپ,4,pes_Arab,2023-06-07
827,https://en.wikivoyage.org/wiki/Paraguay,1,گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شر...,دوران استعمارگری که سه قرن طول کشید توسط کدام ...,پاراگوئه,Guaycurú,گورانی‌ها,فاتحان اسپانیایی,4,pes_Arab,2023-06-07
231,https://en.wikibooks.org/wiki/Wikijunior:Solar...,2,خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید...,با توجه به متن، کدام قسمت از خورشید از زمین قا...,پوسته,پلاسما,فوتوسفر,مرکز,3,pes_Arab,2023-06-07
588,https://en.wikinews.org/wiki/Seven_killed_in_C...,2,آن مرد ظاهراً یک خودروی سه‌چرخ مسلح به مواد من...,مقامات چه اطلاعاتی درخصوص حمله تأیید کرده بودند؟,هویت هم‌دست‌ها,قومیت مظنون,انگیزه حمله,نام و نام خانوادگی مظنون,2,pes_Arab,2023-06-07
39,https://en.wikibooks.org/wiki/Environmental_th...,1,ما می توانیم زندگی دوستانه تری را با محیط زیست...,با توجه به متن، کدام مورد راه‌حلی قابل اجرا بر...,تحریم انتخاب‌های سبک زندگی مصرف‌گرایانه,کنشگری,یافتن ریشه مسئله,تبدیل شدن به متحد جنبش محیط زیست,3,pes_Arab,2023-06-07


In [15]:
os.environ['COHERE_API_KEY'] = 'WaoJWOAwDxfzcjVQWywotLQ8jyxAQc91pyBvXpfq'

# Just predict

**Just predict answer function**

In [16]:
def just_predict(passage, question, options, llm):
    messages = [
        {
            "role": "system",
            "content": (
                "You will receive a `QUESTION` with multiple `ANSWER CHOICES` and a supporting `CONTEXT`. "
                "Your task is to determine the correct answer based on the information in the context.\n\n"
                "Select only one answer choice (1, 2, 3, or 4) based on your understanding of the context.\n"
                "   - Format: <answer>[Your CHOICE here]</answer>"
            )
        },
        {
            "role": "user",
            "content": (
                f"CONTEXT:\n{passage}\n\n"
                f"QUESTION:\n{question}\n\n"
                f"ANSWER CHOICES:\n1-{options[0]}\n2-{options[1]}\n3-{options[2]}\n4-{options[3]}"
            )
        }
    ]
    result = llm.invoke(messages)
    return result.content.strip()

**Predict then Explain**

In [6]:
def explain_then_predict():
    pass

**Explain then Predict**

In [7]:
def predict_then_explain():
    pass

In [17]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Function to check the API with custom parameters: model_name and temperature
def check_api_with_parameters(model_name, temperature):
    llm = ChatCohere(
        model=model_name,
        temperature=temperature,
        max_tokens=None,
        timeout=None,
        max_retries=2
    )
    return llm

# Function to predict the answer and save the full generation to a file
def predict_answer_and_save(passage, question, options, llm, generation_file):
    logging.info(f"Starting prediction for passage: {passage[:50]}... and question: {question}")
    try:
        # Define the messages based on the new prompt format
        messages = [
            {
                "role": "system",
                "content": (
                    "You will receive a `QUESTION` with multiple `ANSWER CHOICES` and a supporting `CONTEXT`. "
                    "Your task is to determine the correct answer based on the information in the context and to explain "
                    "your reasoning step-by-step, referencing key details from the context.\n\n"
                    "Follow these instructions for your response:\n\n"
                    "Select only one answer choice (1, 2, 3, or 4) based on your understanding of the context.\n"
                    "   - Format: <answer>[Your CHOICE here]</answer>\n\n"
                    "Provide a detailed, step-by-step explanation of how you arrived at the answer, focusing on the logical reasoning process. "
                    "Include references to relevant parts of the context to support each reasoning step. `The explanation must be in Persian.`\n"
                    "   - Format: <explanation>[Step-by-step reasoning and references to context here]</explanation>\n\n"
                    "Identify and quote the most relevant parts of the context that were essential in answering the question. "
                    "Only include the specific information that directly supports your chosen answer. `Ensure this is in Persian.`\n"
                    "   - Format: <highlight>[Key supporting context here]</highlight>\n\n"
                    "List the most important keywords from the context that are essential for understanding and answering the question. "
                    "`Provide keywords as a Python list, in Persian.`\n"
                    "   - Format: <keywords>[Relevant KEYWORDS here]</keywords>\n\n"
                    "---\n\n"
                    "`Guidelines for Self-Explanation`:\n"
                    "- Make sure your explanation is complete, covering all reasoning steps used to reach the answer.\n"
                    "- Ensure your reasoning is relevant and directly tied to the context, avoiding any unrelated details.\n"
                    "- Keep the explanation clear and logical, with each step following naturally from the previous one.\n"
                    "- Reflect the information in the context faithfully, without adding unsupported assumptions."
                )
            },
            {
                "role": "user",
                "content": (
                    f"CONTEXT:\n{passage}\n\n"
                    f"QUESTION:\n{question}\n\n"
                    f"ANSWER CHOICES:\n1-{options[0]}\n2-{options[1]}\n3-{options[2]}\n4-{options[3]}"
                )
            }
        ]
        
        # Invoke the language model
        result = llm.invoke(messages)
        
        # Save the full generation to a file with UTF-8 encoding
        with open(generation_file, "a", encoding="utf-8") as f:
            json.dump({
                "Passage": passage,
                "Question": question,
                "Options": options,
                "ModelResponse": result.content.strip()
            }, f, ensure_ascii=False)
            f.write("\n")  # Newline for each entry

        logging.info(f"Full generation saved for parsing: {result.content.strip()}")
        return result.content.strip()
    except Exception as e:
        logging.error(f"Error in predict_answer_and_save: {e}")
        return "error"

# Retry function
def make_request_with_retry(func, *args, retries=3, delay=10):
    for attempt in range(retries):
        try:
            return func(*args)
        except ConnectionError as e:
            logging.warning(f"Attempt {attempt + 1} failed: {e}. Retrying in {delay} seconds...")
            time.sleep(delay)
    logging.error("Max retries exceeded")
    raise ConnectionError("Max retries exceeded")

def apply_prediction_and_save(df, model_name, temperature, generation_file="model_generations.json", mode="just_predict", result_name = 'sample'):
    llm = check_api_with_parameters(model_name, temperature)

    results = []

    for index, row in df.iterrows():
        passage = row['flores_passage']
        question = row['question']
        options = [row['mc_answer1'], row['mc_answer2'], row['mc_answer3'], row['mc_answer4']]
        original_answer = row['correct_answer_num']

        logging.info(f"Processing row {index+1}/{len(df)}: {passage[:50]}...")

        # Select the appropriate method
        if mode == "just_predict":
            predicted_answer = just_predict(passage, question, options, llm)
            time.sleep(7)
        elif mode == "explain_then_predict":
            predicted_answer = explain_then_predict(passage, question, options, llm)
        elif mode == "predict_then_explain":
            predicted_answer = predict_then_explain(passage, question, options, llm)
        else:
            logging.error(f"Invalid mode: {mode}")
            predicted_answer = "error"

        # Save results to the file
        with open(generation_file, "a", encoding="utf-8") as f:
            json.dump({
                "Passage": passage,
                "Question": question,
                "Options": options,
                "OriginalAnswer": original_answer,
                "PredictedAnswer": predicted_answer
            }, f, ensure_ascii=False)
            f.write("\n")

        results.append({
            "Passage": passage,
            "Question": question,
            "Original Answer": original_answer,
            "Predicted Answer": predicted_answer,
            "Option 1": options[0],
            "Option 2": options[1],
            "Option 3": options[2],
            "Option 4": options[3]
        })

        logging.info(f"Row {index+1} processed successfully.")

    # Save final results as DataFrame
    result_df = pd.DataFrame(results)
    output_file_name = result_name + ".csv"
    result_df.to_csv(output_file_name, index=False)
    logging.info("Results saved to {o}".format(o = output_file_name))

    return result_df

In [27]:
# Apply the prediction
apply_prediction_and_save(data, model_name="command-r-plus", temperature=0.3)

2024-12-01 19:30:22,366 - Processing row 1/900: بسیاری، ریتم‌های فیزیولوژی و رفتار را غالباً به صو...
2024-12-01 19:30:23,319 - HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
2024-12-01 19:30:30,322 - Row 1 processed successfully.
2024-12-01 19:30:30,322 - Processing row 2/900: جمعیت عبارت است از مجموعه‌ای از موجودات زنده یک گو...
2024-12-01 19:30:31,231 - HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
2024-12-01 19:30:38,247 - Row 2 processed successfully.
2024-12-01 19:30:38,247 - Processing row 3/900: جمعیت عبارت است از مجموعه‌ای از موجودات زنده یک گو...
2024-12-01 19:30:39,058 - HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
2024-12-01 19:30:46,065 - Row 3 processed successfully.
2024-12-01 19:30:46,066 - Processing row 4/900: جهانگردی طبیعت‌دوست به جذب افراد علاقه‌مند به بازد...
2024-12-01 19:30:47,074 - HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
2024-12-01 19:30:54,079 - Row 4 processed succes

ConnectTimeout: _ssl.c:1105: The handshake operation timed out

In [40]:
rs = pd.read_csv('sample_predicted_answers.csv')

In [41]:
rs.columns

Index(['Passage', 'Question', 'Original Answer', 'Predicted Answer',
       'Option 1', 'Option 2', 'Option 3', 'Option 4'],
      dtype='object')

# Parse Answer

In [42]:
import re

# Function to clean up and normalize Persian text encoding issues
def clean_text(text):
    if text:
        return text.replace('\u200c', '')  # Replace zero-width non-joiners and any encoding artifacts
    return text

# Updated parsing function with robust handling for variations
def parse_prediction(prediction_text):
    # Clean text first to ensure consistency in encoding
    prediction_text = clean_text(prediction_text)
    
    # Patterns to handle different formats and extract components
    answer_pattern = re.compile(r"<answer>(.*?)</answer>|Answer: (\d)", re.DOTALL)
    explanation_pattern = re.compile(r"<(?:explanation|self-explanation)>(.*?)</(?:explanation|self-explanation)>", re.DOTALL)
    highlight_pattern = re.compile(r"<highlight(?:ed)?>\s*(.*?)\s*</highlight(?:ed)?>", re.DOTALL)
    keywords_pattern = re.compile(r"<keywords>\s*\[(.*?)\]\s*</keywords>", re.DOTALL)

    # Extract answer choice, handle possible formats
    answer_match = answer_pattern.search(prediction_text)
    answer = answer_match.group(1).strip() if answer_match and answer_match.group(1) else None

    # Extract explanation with variations
    explanation_match = explanation_pattern.search(prediction_text)
    explanation = explanation_match.group(1).strip() if explanation_match else None

    # Extract highlighted context with variations
    highlight_match = highlight_pattern.search(prediction_text)
    highlight = highlight_match.group(1).strip() if highlight_match else None

    # Extract keywords as a list
    keywords_match = keywords_pattern.search(prediction_text)
    keywords = [k.strip() for k in keywords_match.group(1).split(",")] if keywords_match else []
    
    return {
        "Parsed Answer": answer,
        "Parsed Explanation": explanation,
        "Parsed Highlighted Context": highlight,
        "Parsed Keywords": keywords
    }

In [43]:
# Apply the parsing function to the 'Predicted Answer' column and expand into new columns
parsed_columns = rs['Predicted Answer'].apply(parse_prediction).apply(pd.Series)

# Concatenate the parsed columns with the original DataFrame
df = pd.concat([rs, parsed_columns], axis=1)

In [44]:
df

,Passage,Question,Original Answer,Predicted Answer,Option 1,Option 2,Option 3,Option 4,Parsed Answer,Parsed Explanation,Parsed Highlighted Context,Parsed Keywords
0,بشر هزاران هزار سال از عدسی‌ها برای بزرگ‌نمایی...,کدام یک از موارد زیر را انسان‌ها در قرن شانزده...,4,<answer>4</answer>\n\n<explanation>در متن گفته...,استفاده از عدسی‌ها,بزرگنمایی اشیاء,ساخت عدسی‌ها,ساخت تلسکوپ,4,در متن گفته شده که اولین تلسکوپهای واقعی در قر...,بشر هزاران هزار سال از عدسیها برای بزرگنمایی ا...,"['تلسکوپ', 'عدسی', 'بزرگنمایی', 'قرن ۱۶', 'ارو..."
1,گوارانی‌ها مهم‌ترین گروه بومی ساکن پاراگوئه شر...,دوران استعمارگری که سه قرن طول کشید توسط کدام ...,4,<answer>4</answer>\n\n<explanation>گفته شده اس...,پاراگوئه,Guaycurú,گورانی‌ها,فاتحان اسپانیایی,4,گفته شده است که پاراگوئه در قرن 16ام پس از برخ...,در قرن 16، پاراگوئه که قبلاً «استان غول پیکر ه...,"['گوارانی', 'بومی', 'پاراگوئه', 'شکارچیان', 'ک..."
2,خورشید پوسته‌ای شبیه کره زمین ندارد که بتوانید...,با توجه به متن، کدام قسمت از خورشید از زمین قا...,3,<answer>3</answer>\n\n<explanation>در متن گفته...,پوسته,پلاسما,فوتوسفر,مرکز,3,در متن گفته شده که وقتی به خورشید نگاه میکنیم،...,وقتی به خورشید نگاه میکنیم قسمت بیرونی که میبی...,"['خورشید', 'فوتوسفر']"
3,آن مرد ظاهراً یک خودروی سه‌چرخ مسلح به مواد من...,مقامات چه اطلاعاتی درخصوص حمله تأیید کرده بودند؟,2,<answer>2</answer>\n\n<explanation>در متن گفته...,هویت هم‌دست‌ها,قومیت مظنون,انگیزه حمله,نام و نام خانوادگی مظنون,2,در متن گفته شده که مقامات میدانند مرد مظنون به...,اگرچه آنها میدانند كه او عضو گروه قومی اویغور است,"['مرد مظنون', 'بمبگذاری', 'قومیت', 'اویغور', '..."
4,ما می توانیم زندگی دوستانه تری را با محیط زیست...,با توجه به متن، کدام مورد راه‌حلی قابل اجرا بر...,3,<answer>3</answer>\n\n<explanation>در متن اشار...,تحریم انتخاب‌های سبک زندگی مصرف‌گرایانه,کنشگری,یافتن ریشه مسئله,تبدیل شدن به متحد جنبش محیط زیست,3,در متن اشاره شده است که برای یافتن راهحلی بلند...,«باید ریشه مشکلات را بیابیم و باید آنها را غیر...,"['محیط زیست', 'درمان حمایتی', 'پیشرفتهای علمی'..."
5,ورزش‌های زمستانی روی تپه، از جمله اسکی و اسنوب...,مطابق با متن، حدود چه زمانی اسکی‌بازی اولین با...,4,<answer>4</answer>\n\n<explanation>متن اشاره م...,قرن هجدهم,5000 سال قبل از میلاد,1861,قرن هفدهم,4,متن اشاره میکند که ورزش اسکی روی تپه «حداقل» ب...,ورزش اسکی روی تپه حداقل به قرن 17 ام برمیگردد,"[""ورزشهای زمستانی"", ""اسکی"", ""اسنوبورد"", ""تپه"",..."
6,ACMA همچنین نتیجه گرفت که علیرغم اینکه این وید...,برای اینکه رسانه تحت مقوله محتوای اینترنتی لحا...,2,<answer>2</answer>\n\n<explanation>در متن گفته...,باید از طریق اینترنت پخش شود,باید به صورت فیزیکی در سروری قرار گیرد,نمی‌تواند روی وبسایت ذخیره شود,باید توسط ACMA تأیید شود,2,"در متن گفته میشود که محتوا برای اینکه جزء ""محت...",قانون خدمات پخش مقررات محتوای اینترنتی را مشخص...,"['قانون خدمات پخش', 'محتوای اینترنتی', 'سرور',..."
7,عرفان عبارت است از جستجوی ارتباط، شناخت یا درک...,با توجه به متن، کدام یک از موارد زیر مورد توجه...,2,<answer>2</answer>\n\n<explanation>عرفان به دن...,جستجوی شهود به واقعیت الهی,طلب بینش از طریق تجربیات مشترک و همگانی,تأکید بر حالت سعادتمند خودآگاه,طلب درک آگاهانه الوهیت,2,عرفان به دنبال تجربه مستقیم و شخصی از واقعیت ا...,عرفان عبارت است از جستجوی ارتباط، شناخت یا درک...,"['عرفان', 'تجربه مستقیم', 'واقعیت الهی', 'بینش..."
8,محمد عمیقا به امور فراتر از این زندگی دنیوی عل...,بر اساس متن، غار محمد بیانگر چه چیزی درباره اع...,1,<answer>1</answer>\n\n<explanation>غار حرا، مک...,او برای آرامش و انزوا ارزش قائل بود,صدای بیرون به تأمل او کمک می‌کرد,به زمان خیلی کمی برای تنهایی نیاز داشت,نگران این بود که دیده یا شنیده شود,1,غار حرا، مکانی بود که محمد برای تأمل و تفکر ان...,خود غار که طی اعصار سالم مانده است بیانگر تصور...,"[""غار حرا""، ""تأمل""، ""کوه نور""، ""انزوا""، ""آرامش..."
9,کودکان از زمان کودکی نسبت به نژاد و کلیشه های ...,با توجه به نظریه ارائه شده در متن، برای کودکان...,2,<answer>1</answer>\n\n<explanation>با توجه به ...,کودک سعی می‌کند از درگیر شدن در رفتارهای تهدید...,احتمال بیشتری وجود دارد که کودک درگیر رفتارهای...,کودک در موقعیت‌های اجتماعی کناره‌گیری می‌کند,کودک از تعامل با سایر اقلیت‌های نژادی امتناع خ...,1,

In [45]:
from IPython.display import display, HTML

# Function to display each result with RTL formatting for Passage and Question
def display_result(row):
    html = f"""
    <div style='border: 2px solid black; padding: 10px; margin-bottom: 10px;'>
        <p><strong>Row {row.name + 1}:</strong></p>
        <p><strong>Passage:</strong> <span style='direction: rtl; text-align: right; display: block;'>{row['Passage']}</span></p>
        <p><strong>Question:</strong> <span style='direction: rtl; text-align: right; display: block;'>{row['Question']}</span></p>
        <p><strong>Original Answer:</strong> {row['Original Answer']}</p>
        <p><strong>Predicted Answer:</strong> {row['Parsed Answer']}</p>
        <p><strong>Options:</strong>
            <br>1: {row['Option 1']}
            <br>2: {row['Option 2']}
            <br>3: {row['Option 3']}
            <br>4: {row['Option 4']}
        </p>
        <p><strong>Explanation:</strong> <span style='direction: rtl; text-align: right; display: block;'>{row['Parsed Explanation']}</span></p>
        <p><strong>Highlighted Context:</strong> <span style='direction: rtl; text-align: right; display: block;'>{row['Parsed Highlighted Context']}</span></p>
        <p><strong>Keywords:</strong> {row['Parsed Keywords']}</p>
    </div>
    """
    display(HTML(html))

# Apply the display function to all rows in the DataFrame
for _, row in df.iterrows():
    display_result(row)

In [46]:
df.to_csv('resulttt.csv')